In [20]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
from urllib.parse import quote

In [2]:
#defining functions for amazon and flipkart 
def get_title(soup):
    try:
        title=soup.find('h1', class_="yhB1nd").get_text(strip=True)
        if title:
            return title
    except AttributeError:
        pass
    return 'NA'
    
def get_price(soup):
    try:
        price= soup.find('div', class_='_30jeq3 _16Jk6d').get_text(strip=True)
        if price:
            return price
    except AttributeError:
        pass
    return 'NA'
def get_ratings(soup):
    try:
        ratings= soup.find('div', class_='_3LWZlK').get_text(strip=True)
        if ratings:
            return ratings
    except AttributeError:
        pass
    return 'NA'
    
def get_author(soup):
    try:
        author=soup.find('a',class_='_1mXcCf RmoJUa _14Me7y').text
        if author:
            return author
    except AttributeError:
        pass
    return 'NA'
def get_website_f():
    return 'Flipkart'  

In [3]:
#functions for amazon 
def get_price_A(soup):
    try:
        price_element = soup.find('span', {'class': 'a-size-base a-color-price a-color-price'})
        if price_element:
            price = price_element.get_text(strip=True)
            return price
    except AttributeError:
        pass  

    return 'NA'

def get_website_a():
    return 'Amazon'
def get_title_A(soup):
    try:
        title=soup.find('span', id ='productTitle').get_text(strip=True)
        if title:
            return title
    except AttributeError:
        pass
    return 'NA'
def get_ratings_A(soup):
    try:
        ratings= soup.find('span',{'id':'acrCustomerReviewText'}).text
        if ratings:
            return ratings
    except AttributeError:
        pass
    return 'NA'
def get_author_A(soup):
    try:
        author_span = soup.find('span', {'class': 'author notFaded'}).get_text(strip=True)
        author = author_span.replace('(Author),', '').strip()
        if author:
            return author
    except AttributeError:
        pass
    return 'NA'

In [5]:
data_df= pd.read_csv('Website_data2.csv')
data_df

,Website,Item
0,Amazon,ML book
1,Amazon,DBMS book
2,Flipkart,ML book
3,Flipkart,DBMS book


In [10]:
def get_item():
    if (data_df['Item']=='ML book').any():
        return 'ML book'
    else:
        return 'DBMS book'
def get_item2():
    if (data_df['Item']=='DBMS book').any():
        return 'DBMS book'
    
get_item()
get_item2()

'DBMS book'

In [25]:
# defining function for scraping
def scrape_data(item, base_url, headers, class_selector, get_title, get_author, get_price, get_web, get_rat):
    encoded_item = quote(item)
    url = base_url.format(encoded_item)
    print("Constructed URL:", url)  
    webpage = requests.get(url, headers=headers) 
    soup = BeautifulSoup(webpage.content, "html.parser")
    links = soup.find_all('a', class_=class_selector)
    book_list = []

    for link in links:
        absolute_url = link.get('href')
        full_url = urljoin("https://www.amazon.in", absolute_url)
        print("Appended URL:", full_url) 
        book_list.append(full_url)
        time.sleep(1)

    data = {'Book_name': [], 'Book_price': [], 'Book_ratings': [], 'Book_Author': [], 'Website': []}

    for link in book_list:
        new_page = requests.get(link, headers=headers)
        new_soup = BeautifulSoup(new_page.content, "html.parser")
        Book_name = get_title(new_soup)
        Book_price = get_price(new_soup)
        Book_ratings = get_rat(new_soup)
        Book_Author = get_author(new_soup)
        Book_website = get_web()

        data['Book_name'].append(Book_name)
        data['Book_price'].append(Book_price)
        data['Book_Author'].append(Book_Author)
        data['Book_ratings'].append(Book_ratings)
        data['Website'].append(Book_website)
        time.sleep(1)

    return data

base_amazon_url = 'https://www.amazon.in/s?k={}'
#website="https://www.amazon.in"
combined_data = []
for i, row in data_df.iterrows():
    website = row['Website']
    item = row['Item']
    if website == 'Amazon' and item in ['ML book', 'DBMS book']:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'
        }
        class_selector = "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"
        data = scrape_data(item, base_amazon_url, headers, class_selector, get_title_A, get_author_A, get_price_A, get_website_a, get_ratings_A)
        combined_data.append(data)

combined_df = pd.concat([pd.DataFrame(data) for data in combined_data], ignore_index=True)
combined_df.to_csv('Website_Final_Automated.csv', header=True, index=False)


Constructed URL: https://www.amazon.in/s?k=ML%20book
Appended URL: https://www.amazon.in/sspa/click?ie=UTF8&spc=MToyODg2NzA4NjU4MTA2MDU1OjE3MDg4OTM0MjI6c3BfYXRmOjIwMDg2MzQ3NTg5MjAxOjowOjo&url=%2FAccelerated-DevOps-AI-RPA-Non-Programmers%2Fdp%2F1647130514%2Fref%3Dsr_1_1%3Fdib%3DeyJ2IjoiMSJ9.4HQmnSCa-qY94V6j3FNxy3DKdU6PE-l62zTgTxanrb7ekie1sFO0f4ziLMg9e09TmBc6DdFTfWAWQeVMicygzMKYxyKWa3t3eQQQy7WwJ6uwsfIVAUqtxqTUI31zXK4aFEH57f6h2yDV-IT3_UkyTBy4oU15580G_07RQgvqxPaGiHbfbzaerbwPn5sbjeBtDZvhQfmJLYGnkjI449VSzq56ksd4D6hiykai47gkREU.q5vdjn8MWRwsvCFrmfOSInfnlXw8LwP-e7u5pZ5giQw%26dib_tag%3Dse%26keywords%3DML%2Bbook%26qid%3D1708893422%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
Appended URL: https://www.amazon.in/Introduction-AI-ML-Data-Science/dp/9356684162/ref=sr_1_2?dib=eyJ2IjoiMSJ9.4HQmnSCa-qY94V6j3FNxy3DKdU6PE-l62zTgTxanrb7ekie1sFO0f4ziLMg9e09TmBc6DdFTfWAWQeVMicygzMKYxyKWa3t3eQQQy7WwJ6uwsfIVAUqtxqTUI31zXK4aFEH57f6h2yDV-IT3_UkyTBy4oU15580G_07RQgvqxPaGiHbfbzaerbwPn5sbjeBtDZvhQfmJ

In [26]:
combined_df

,Book_name,Book_price,Book_ratings,Book_Author,Website
0,"Accelerated DevOps with AI, ML & RPA",₹748.00,9 ratings,Stephen Fleming(Author),Amazon
1,Introduction to AI/ML and Data Science,₹450.00,6 ratings,Sudhir Reddy Navar(Author),Amazon
2,Natural Language Processing (NLP) and Machine ...,"₹5,553.00",NA,"Florentina Hristea(Editor),",Amazon
3,Machine Learning Interview Questions and Answe...,₹100.00,5 ratings,Aditya Chatterjee,Amazon
4,The Economics Of Development And Planning By M...,₹440.00,NA,ML Jhingan(Author),Amazon
5,Hands-On Artificial Intelligence on Amazon Web...,"₹2,980.00",6 ratings,"Subhashini Tripuraneni(Author, Contributor)",Amazon
6,The Handbook for AI and ML : The Only Handbook...,NA,2 ratings,Steno Wickroth(Author),Amazon
7,Designing Machine Learning Systems: An Iterati...,"₹1,600.00",494 ratings,Chip Huyen(Author),Amazon
8,Sahitya Bhawan Samajshastra book by ML Gupta D...,₹840.00,162 ratings,M. L. Gupta,Amazon
9,Understanding ISC Mathematics Vol 1& 2 Class 1...,"₹1,500.00",NA,ML Aggarwal(Author),Amazon
